Experiment 1

Setting up environment

In [1]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')


    basepath = '/content/drive/MyDrive'

else:
    basepath = '/home/harpreet/Insync/google_drive_shaannoor/data'


Mounted at /content/drive


Load Libraries

In [2]:
pip install evaluate datasets transformers wandb huggingface_hub

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 472.7/472.7 kB 18.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 16.1 MB/s eta 0:00:00


In [3]:
# standard data science librraies for data handling and v isualization
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from sklearn.metrics import confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

from sklearn.preprocessing import MultiLabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import joblib


# New libraries introduced in this notebook
import evaluate
from datasets import load_dataset, DatasetDict, Dataset, ClassLabel
from transformers import TrainingArguments, Trainer, set_seed
from transformers import AutoTokenizer
from transformers import AutoModelForSequenceClassification
from transformers import AutoConfig
from transformers import pipeline
import wandb
from google.colab import userdata
from huggingface_hub import login
import torch.nn as nn
import ast

In [6]:
import torch
from transformers import RobertaTokenizer, RobertaForSequenceClassification

In [7]:
# Initialize Weights & Biases (Replace "your_project_name" with your W&B project name)
wandb.init(project="Exp3")

wandb: Using wandb-core as the SDK backend. Please refer to https://wandb.me/wandb-core for more information.


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [8]:
# CHANGE FOLDERS TO WHERE YOU WANT TO SAVE DATA AND MODELS
base_folder = Path(basepath)
data_folder = base_folder/'datasets/HW6/Datafolder'

Load Dataset

In [9]:
train_df = pd.read_csv(data_folder/'train.csv')
test_df = pd.read_csv(data_folder/'test.csv')
sample_submission = pd.read_csv(data_folder/'sample_submission.csv')

In [10]:
pip install emoji

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 586.9/586.9 kB 11.6 MB/s eta 0:00:00


In [11]:
# Define a function to clean the tweets
import re
import emoji

def clean_tweet(tweet):
    tweet = tweet.lower()
    tweet = re.sub(r'http\S+|www\S+|https\S+', '', tweet)
    tweet = re.sub(r'@\w+', '', tweet)
    tweet = re.sub(r'#', '', tweet)
    tweet = re.sub(r'\d+', '', tweet)
    tweet = re.sub(r'[^\w\s]', '', tweet)
    tweet = emoji.demojize(tweet, delimiters=(" ", " "))
    return tweet

train_df['Cleaned_Tweet'] = train_df['Tweet'].apply(clean_tweet)
test_df['Cleaned_Tweet'] = test_df['Tweet'].apply(clean_tweet)

In [12]:
# Preprocess labels in test_df to be integers (all 0s)
for emotion in ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']:
    test_df[emotion] = 0

In [13]:
label_encoder = MultiLabelBinarizer()

In [14]:
# Keep only the 'Tweet' (as 'text') and 'label' columns
train_df = train_df[['Cleaned_Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].rename(columns={'Cleaned_Tweet': 'Tweet'})
test_df = test_df[['Cleaned_Tweet', 'anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']].rename(columns={'Cleaned_Tweet': 'Tweet'})


Tokenizer

In [15]:
# Use RobertaTokenizer for distilroberta-base
tokenizer = RobertaTokenizer.from_pretrained("distilroberta-base")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/25.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/899k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/480 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


Custom dataset

In [16]:
# Define a custom dataset class
class EmotionDataset(torch.utils.data.Dataset):
    def __init__(self, texts, labels, tokenizer, max_length=128):
        self.texts = texts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.texts)

    def __getitem__(self, idx):
        inputs = self.tokenizer(self.texts[idx], truncation=True, padding='max_length', max_length=self.max_length, return_tensors="pt")
        inputs = {key: val.squeeze() for key, val in inputs.items()}
        inputs["labels"] = torch.tensor(self.labels[idx], dtype=torch.float)
        return inputs

Data preparation

In [17]:
# Calculate class weights based on dataset distribution
emotion_columns = ['anger', 'anticipation', 'disgust', 'fear', 'joy', 'love', 'optimism', 'pessimism', 'sadness', 'surprise', 'trust']
emotion_distribution = train_df[emotion_columns].sum()
total_samples = len(train_df)
class_weights = torch.tensor([total_samples / count for count in emotion_distribution])

train_texts, val_texts, train_labels, val_labels = train_test_split(train_df['Tweet'], train_df[emotion_columns].values, test_size=0.2)
train_dataset = EmotionDataset(train_texts.tolist(), train_labels.tolist(), tokenizer)
val_dataset = EmotionDataset(val_texts.tolist(), val_labels.tolist(), tokenizer)

In [18]:
class WeightedDistilRobertaForSequenceClassification(RobertaForSequenceClassification):
    def __init__(self, config, weights):
        super().__init__(config)
        self.weights = weights

    def forward(self, input_ids=None, attention_mask=None, labels=None, **kwargs):
        outputs = super().forward(input_ids=input_ids, attention_mask=attention_mask, labels=labels, **kwargs)
        logits = outputs.logits

        if labels is not None:
            # Applying class weights to the loss function
            loss_fct = nn.BCEWithLogitsLoss(pos_weight=self.weights.to(logits.device))
            loss = loss_fct(logits, labels)
            outputs = (loss,) + outputs[1:]

        return outputs

Model Training with W&B Logging

In [19]:
# Instantiate the model with weights
model = WeightedDistilRobertaForSequenceClassification.from_pretrained(
    "distilroberta-base",
    num_labels=len(emotion_columns),
    weights=class_weights
)

model.safetensors:   0%|          | 0.00/331M [00:00<?, ?B/s]

Some weights of WeightedDistilRobertaForSequenceClassification were not initialized from the model checkpoint at distilroberta-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [20]:
# Define metrics calculation function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = (pred.predictions > 0.5).astype(int)  # Apply threshold for binary predictions
    f1 = f1_score(labels, preds, average="macro")
    accuracy = accuracy_score(labels, preds)
    return {"f1": f1, "accuracy": accuracy}

In [21]:
# Training arguments
training_args = TrainingArguments(
    output_dir='./results',
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir='./logs',
    report_to="wandb"  # Enable W&B logging
)


/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [22]:
# Trainer setup with class weights and metrics
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    compute_metrics=compute_metrics
)

In [23]:
# Train and log results
trainer.train()

wandb: WARNING The `run_name` is currently set to the same value as `TrainingArguments.output_dir`. If this was not intended, please specify a different run name by setting the `TrainingArguments.run_name` parameter.


Epoch,Training Loss,Validation Loss,F1,Accuracy
1,1.014400,0.876124,0.548893,0.131392
2,0.772300,0.855394,0.558277,0.161165
3,0.655000,0.851836,0.570365,0.150809


TrainOutput(global_step=2319, training_loss=0.7958840275180911, metrics={'train_runtime': 351.8458, 'train_samples_per_second': 52.685, 'train_steps_per_second': 6.591, 'total_flos': 613985572456704.0, 'train_loss': 0.7958840275180911, 'epoch': 3.0})

In [24]:
trainer.evaluate()

{'eval_loss': 0.8518358469009399,
 'eval_f1': 0.570364868640934,
 'eval_accuracy': 0.15080906148867315,
 'eval_runtime': 5.9822,
 'eval_samples_per_second': 258.267,
 'eval_steps_per_second': 32.43,
 'epoch': 3.0}

Generate Predictions for Submission

In [25]:
# Move model to device (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# Predict on test set
test_texts = test_df['Tweet'].tolist()
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128, return_tensors="pt").to(device)

# Generate predictions
with torch.no_grad():
    outputs = model(**test_encodings)
predictions = torch.sigmoid(outputs.logits).cpu().numpy()

In [26]:
# Prepare submission file
submission = sample_submission.copy()
submission[emotion_columns] = (predictions > 0.5).astype(int)  # Threshold at 0.5 for binary classification
submission.to_csv("submission_distilroberta.csv", index=False)

In [27]:
from google.colab import files
# Download the saved submission file
files.download("submission_distilroberta.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>